In [27]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import numpy as np
from shutil import copyfile
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
# Image ids need to be numbers
# Collect image names from each split of images
names = []
path = Path('PeopleArt/')
for split in ['train', 'val', 'test']:
    for img_path in path.rglob(split+'/*.jpg'):
        fname = img_path.as_posix().split('/')[-1]
        img_name = fname.split('.')[0]
        names.append(img_name)

# Create look up dictionary, stored as .npy, mapping image name : id
lookup = {name : str(idx) for name, idx in zip(names, range(len(names)))}
# print(list(lookup.items())[4622])
np.save('img_name2int', lookup)

# Go thru XMLs checking if name in dict, then copying + renaming to id
for split in ['train', 'val', 'test']:
    new_dir = Path.cwd() / 'PeopleArt' / (split + '_int_xmls')
    for xml_path in path.rglob(split+'/*.xml'):
        fname = xml_path.as_posix().split('/')[-1]
        xml_name = fname.split('.')[0]
        if xml_name in lookup:
            new_fname = lookup[xml_name]
            copyfile(xml_path , new_dir / (new_fname + '.jpg.xml') )

In [13]:
!find ./PeopleArt/train_int_xmls > ann_paths_train.txt -type f
!find ./PeopleArt/val_int_xmls > ann_paths_val.txt -type f
!find ./PeopleArt/test_int_xmls > ann_paths_test.txt -type f

In [7]:
# Run VOC to COCO conversion on each split of XMLs
!python voc2coco.py \
    --ann_paths_list ann_paths_train.txt \
    --labels labels.txt \
    --output train.json \
    --ext xml

Start converting !
100%|███████████████████████████████████████| 525/525 [00:00<00:00, 9774.63it/s]


In [8]:
!python voc2coco.py \
    --ann_paths_list ann_paths_val.txt \
    --labels labels.txt \
    --output val.json \
    --ext xml

Start converting !
100%|███████████████████████████████████████| 443/443 [00:00<00:00, 9451.82it/s]


In [9]:
!python voc2coco.py \
    --ann_paths_list ann_paths_test.txt \
    --labels labels.txt \
    --output test.json \
    --ext xml

Start converting !
100%|██████████████████████████████████████| 522/522 [00:00<00:00, 10681.02it/s]


In [35]:
# Copy and rename each img with a XML annotation to a new split_int_imgs folder
lookup = np.load('img_name2int.npy').item()
# print(type(lookup))

# Go thru XMLs checking if name in dict, then copying + renaming to id
path = Path('PeopleArt/')
for split in ['train', 'val', 'test']: 
    curr_img_dir = Path.cwd() / 'PeopleArt' / split
    new_dir = Path.cwd() / 'PeopleArt' / (split + '_int_imgs')
    for xml_path in path.rglob(split+'/*.xml'):
        # get image name from XML file path
        fname = xml_path.as_posix().split('/')[-1]
        img_jpg = os.path.splitext(fname)[0] # removes .xml, includes .jpg # fname.split('.')[0]
        img_name = fname.split('.')[0] # os.path.splitext(img_jpg)[0]
        # TODO bad hygiene for lookup dict path splitting. some file names have website.com in them. 
        # should split by extension from the end, not take first element of split('.') 
        
        # copy corresponding image to new directory
        img_path = curr_img_dir / img_jpg
        new_fname = lookup[img_name]
        copyfile(img_path , new_dir / (new_fname + '.jpg') )

In [38]:
!ls PeopleArt/train_int_imgs | wc -l
!ls PeopleArt/val_int_imgs | wc -l
!ls PeopleArt/test_int_imgs | wc -l

525
443
522
